<img src='images/union-find.png' width=600>

In [62]:
# n = 5
# Expected result:
# For K = 2 -> 8
# For K = 3 -> 4
# For K = 4 -> 1

test1 = [
    (1, 2, 1),
    (1, 3, 4),
    (1, 4, 5),
    (1, 5, 10),
    (2, 3, 5),
    (2, 4, 4),
    (2, 5, 8),
    (3, 4, 1),
    (3, 5, 12),
    (4, 5, 11)
]

In [63]:
from collections import defaultdict
def convert_graph(lst):
    graph = defaultdict(dict)
    for t in lst:
        graph[t[0]][t[1]] = t[2]
        graph[t[1]][t[0]] = t[2]
    return graph

In [74]:
class UnionFind:
    def __init__(self, n):
        self.node = n
        self.leader = self
        self.members = [self]
    def __repr__(self):
        return '<UnionFind {}>'.format(self.node)
    

def union(uf1, uf2, s):  # s: remove node from set
    assert isinstance(uf1, UnionFind) and isinstance(uf2, UnionFind), 'Not instance of UnionFind' 
    # cluster that has more members will keep leader point
    # whereas cluster with fewer members will update leader point
    # if number of members eqal
    # the node with smaller value keep leader point
    
    if uf1.leader is uf2.leader:
        return 

    if len(uf1.leader.members) <= len(uf2.leader.members):
        s.remove(uf1.leader.node)
        uf2.leader.members.extend(uf1.leader.members)
        for uf in uf1.leader.members:
            uf.leader = uf2.leader
            uf.members = []
    else:
        s.remove(uf2.leader.node)
        uf1.leader.members.extend(uf2.leader.members)
        for uf in uf2.leader.members:
            uf.leader = uf1.leader
            uf.members = []
                

def cluster(lst, k):
    graph = convert_graph(lst)
    nodes = set(graph.keys())
    
    lst.sort(key=lambda x: x[2])
    ufs = {n: UnionFind(n) for n in nodes}
    
    i = 0
    cur_space = 99999
    while len(nodes) > k:
        edge = lst[i]
        print(edge)
        cur_space = edge[2]  # cost of edge
        union(ufs[edge[0]], ufs[edge[1]], nodes)
        i += 1

    return ufs, nodes, lst[i:]


def max_space():
    pass

In [76]:
ufs, nodes, lst = cluster(test1, 2)
ufs, nodes, lst 


(1, 2, 1)
(3, 4, 1)
(1, 3, 4)


({1: <UnionFind 1>,
  2: <UnionFind 2>,
  3: <UnionFind 3>,
  4: <UnionFind 4>,
  5: <UnionFind 5>},
 {4, 5},
 [(2, 4, 4),
  (1, 4, 5),
  (2, 3, 5),
  (2, 5, 8),
  (1, 5, 10),
  (4, 5, 11),
  (3, 5, 12)])

In [67]:
ufs[2].leader

<UnionFind 2>

In [31]:
sorted(test1, key=lambda x: x[2])

[(1, 2, 1),
 (3, 4, 1),
 (1, 3, 4),
 (2, 4, 4),
 (1, 4, 5),
 (2, 3, 5),
 (2, 5, 8),
 (1, 5, 10),
 (4, 5, 11),
 (3, 5, 12)]

In [63]:
# 5, 5
# Expected result: 2

test2 = [
    (0, 0, 0, 0, 0),
    (1, 1, 1, 1, 1),
    (0, 0, 0, 0, 1),
    (1, 1, 1, 0, 0),
    (0, 0, 0, 1, 0)
]